In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader

from mingpt.utils import set_seed
from mingpt.model import GPT
from mingpt.trainer import Trainer

c:\Users\dalli\anaconda3\envs\pydl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set model hyperparameters
select_mingpt = True
model_architecture = 'gpt2'
device = 'cpu'
set_seed(42)

Dataset

In [ ]:
# load data
dataset = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", "plain_text", cache_dir="datasets")["train"]

In [ ]:
# dataset class
class RedPajama(Dataset):
    def __init__(self, data, max_length=1024):
        # super().__init__()
        self.data = data
        self.vocab_size = max_length
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_architecture)
        self.tokenizer.pad_token_id = 50256
        self.vocab_size = self.tokenizer.vocab_size

    def __len__(self):
        # return the number of data points
        return len(self.data)
    
    def __getitem__(self, idx):
        # load and tokenize the text
        text = self.data[idx]["text"]
        tokens = self.tokenizer.encode(text, 
                                       add_special_tokens=True, 
                                       max_length=self.max_length,
                                       truncation=True,
                                       return_tensors="pt",
                                       padding=True)
        
        # split and shift tokens after excluding the CLS token
        targets = tokens[:, 1:].clone()
        tokens = tokens[:, :-1].clone()

        return tokens, targets

rp_dataset = RedPajama(dataset)

In [ ]:
# set up data path
path = "./checkpoints/redpajama"

if not os.path.exists(path):
    os.makedirs(path)
    checkpoints = os.listdir(path)
else:
    checkpoints = os.listdir(path)
checkpoints.sort()

Model

In [ ]:
# set up model configurations
model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = rp_dataset.get_vocab_size()
model_config.block_size = rp_dataset.get_block_size()
model_config.checkpoint = 'checkpoints/' + checkpoints[-1] if checkpoints else None

# set up model with configurations
model = GPT(model_config)

In [ ]:
# set up trainer configurations
max_iters = 500
train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = max_iters + model.iter_num if model_config.checkpoint else max_iters
train_config.num_workers = 0
train_config.checkpoint_iters = 100 
train_config.batch_size = 1
train_config.checkpoint_name = path

# set up trainer with configurations
trainer = Trainer(train_config, model, rp_dataset)

In [ ]:
# train model
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

In [ ]:
# show loss
plt.plot(trainer.iter_list, [a.detach() for a in trainer.saved_loss])
plt.show()

Use Model

In [ ]:
def generate(prompt="", sample_count=10, steps=20, do_sample=True):
    # handle empty prompt
    if prompt == "": prompt = "<|endoftext|>"
    
    # tokenize and encode the prompt in batches
    tokenizer = GPT2Tokenizer.from_pretrained(model_architecture)
    tokenized = tokenizer(prompt, return_tensors="pt").to(device)
    input = tokenized["input_ids"].expand(sample_count, -1)

    # generate output
    output = model.generate(input, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    for i in range(sample_count):
        print(f"Sample {i+1}: {tokenizer.decode(output[i].cpu().squeeze(), skip_special_tokens=True)}\n")

In [ ]:
generate("Math is")